In [2]:
import numpy as np
import pandas as pd

→ Проектирование признаков — самый творческий процесс во всём цикле жизни создания модели машинного обучения. 

Он включает в себя несколько этапов: 

- создание признаков;
- преобразование признаков;
- отбор признаков.

In [3]:
data = pd.read_csv('data/wine_cleared.csv')

In [4]:
# для удобства сразу преобразуем признак в int
data['price_round'] = data['price'].round().astype(int)

In [5]:
regex = '\d{4}' # регулярное выражение для нахождения чисел
data['year'] = data['title'].str.findall(regex).str.get(0)

РАЗБОР КАТЕГОРИЙ

В наборе данных винных обзоров самая популярная страна-производитель вина — США. Возможно, это не случайность, и факт производства в США влияет на рейтинг вина. Выделим этот факт.

Вы можете создать новый бинарный признак is_usa и присвоить ему 1 в случае, если вино произведено в США, иначе — 0.

Создайте бинарные признаки is_france, is_italy наподобие признака is_usa.

In [6]:
data['is_usa'] = data['country'].apply(lambda x: 1 if x == 'US' else 0)
data['is_france'] = data['country'].apply(lambda x: 1 if x == 'France' else 0)
data['is_italy'] = data['country'].apply(lambda x: 1 if x == 'Italy' else 0)

In [7]:
data['is_italy'].sum()

19540

In [8]:
data['old_wine'] = data['year'].apply(lambda x: 1 if x == 'Italy' else 0)

In [9]:
def intur(x):
    if x is np.nan:
        return x
    result = int(x)
    return result
data['new_year'] = data['year'].apply(intur)

In [10]:
data['old_wine'] = data['new_year'].apply(lambda x: 1 if x < 2010 else 0)

In [11]:
data['old_wine'].sum()

39781

In [21]:

regex = '.*(a-z).*'# регулярное выражение для нахождения чисел
data['locality'] = data['title'].str.findall(regex).str.get(0)

In [22]:
data['locality'].value_counts()

Series([], Name: locality, dtype: int64)

РАБОТА С ФАЙЛАМИ

Часто маленькие страны с небольшим количеством населения имеют узкую специализацию. Например, в производстве вина особенно успешны Франция, Италия, Испания, Новая Зеландия. Чтобы проверить, влияет ли на качество вина населённость, выясним информацию о населении страны, в котором была произведена бутылка вина. 

In [41]:
country_population = pd.read_csv('data/country_population.csv', sep=';')
country_population.head(10)



,country,population
0,China,"1,411,778,724"
1,India,"1,386,584,581"
2,US,"333,022,386"
3,Indonesia,"271,350,000"
4,Pakistan,"225,200,000"
5,Brazil,"214,191,924"
6,Nigeria,"211,401,000"
7,Bangladesh,"172,011,942"
8,Russia,"146,171,015"
9,Mexico,"126,014,024"


In [42]:
country_population[country_population['country'] == 'Italy']['population']

24    59,097,904
Name: population, dtype: object

Далее сопоставим значения из датасета country_population и страной-производителем вина. На основе значений населения из country_population заполним новый признак country_population.

Используем для этого функцию для объединения датасетов join. Для объединения используем аргумент on='country', указывая столбец, по которому объединяем датафреймы:

In [ ]:
data.join(country_population.set_index('country'), on='country')

In [43]:
country_area = pd.read_csv('data/country_area.csv', sep=';')
country_area

,country,area
0,Russia,17075400.00
1,Canada,9984670.00
2,China,9596960.00
3,US,9372610.00
4,Brazil,8547000.00
...,...,...
188,San Marino,61.00
189,Tuvalu,26.00
190,Nauru,21.00
191,Monaco,2.00


In [62]:
new_data = data.join(country_area.set_index('country'), on='country')

In [70]:
new_data[new_data['title'] == 'Gård 2014 Grand Klasse Reserve Lawrence Vineyards Viognier (Columbia Valley (WA))']['area']

94    9372610.0
Name: area, dtype: float64